<a href="https://colab.research.google.com/github/aryaar04/Sentiment-Analysis-Dashboard-for-Twitter-Reddit-Data/blob/main/Sentiment_Analysis_Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sentiment Analysis Dashboard for Twitter/Reddit Data

In [10]:
# Required installations (if not already installed)
# !pip install tweepy plotly nltk pandas

import tweepy
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import datetime

# Download VADER lexicon if not already
nltk.download('vader_lexicon')

# Twitter API credentials (replace with your own)
bearer_token = "AAAAAAAAAAAAAAAAAAAAAIS90wEAAAAACBJRi2NsVg81lpF8IxO1ODs8HQ4%3DBCUh36VSJSi45e9AImyC1gzTI9q4TKWUikAtithpb10voPrwTi"

# Initialize Twitter API client
client = tweepy.Client(bearer_token=bearer_token)

# Initialize sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Function to fetch tweets
def fetch_tweets(query, max_results=100):
    tweets = []
    try:
        response = client.search_recent_tweets(
            query=query + " -is:retweet lang:en",
            max_results=min(max_results, 100),
            tweet_fields=["created_at", "text"]
        )
        for tweet in response.data:
            tweets.append(tweet.text)
    except Exception as e:
        print(f"Error: {e}")
    return tweets

# Function to analyze sentiments
def analyze_sentiments(tweets):
    results = {"Tweet": [], "Sentiment": [], "Score": []}
    for tweet in tweets:
        score = sia.polarity_scores(tweet)['compound']
        if score >= 0.05:
            sentiment = "Positive"
        elif score <= -0.05:
            sentiment = "Negative"
        else:
            sentiment = "Neutral"
        results["Tweet"].append(tweet)
        results["Sentiment"].append(sentiment)
        results["Score"].append(score)
    return pd.DataFrame(results)

# Main function
def sentiment_dashboard():
    query = input("Enter a keyword or topic for sentiment analysis: ")
    tweets = fetch_tweets(query, max_results=100)

    if not tweets:
        print("No tweets found.")
        return

    df = analyze_sentiments(tweets)
    sentiment_counts = df['Sentiment'].value_counts().reset_index()
    sentiment_counts.columns = ['Sentiment', 'Count']

    # Pie chart
    fig1 = px.pie(sentiment_counts, names='Sentiment', values='Count', title=f"Sentiment Distribution for '{query}'")
    fig1.show()

    # Bar chart
    fig2 = px.bar(sentiment_counts, x='Sentiment', y='Count', color='Sentiment', title='Sentiment Count')
    fig2.show()

    # Line chart (optional - visualize score trend)
    df['Index'] = df.index
    fig3 = px.line(df, x='Index', y='Score', title='Sentiment Score Trend')
    fig3.show()

    print("\nSample Analyzed Tweets:")
    print(df[['Tweet', 'Sentiment', 'Score']].head())

# Run the dashboard
sentiment_dashboard()


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Enter a keyword or topic for sentiment analysis: water



Sample Analyzed Tweets:
                                               Tweet Sentiment   Score
0  Macquarie ‘very proud’ of Thames Water ownersh...  Negative -0.6908
1  ⠀\nhow was she supposed to\nknow there was a c...  Positive  0.6249
2  In heavy rain and floods, your #tyres struggle...  Negative -0.7003
3  @k59frxsqr2 I should give you a medal 🏅 buddy....  Positive  0.4767
4  "Sleek and stylish ✨ Our Copper Bottle with a ...  Positive  0.7906


In [8]:
# Install the Hugging Face transformers library
!pip install -q transformers

# Import the pipeline module
from transformers import pipeline

# Initialize the sentiment analysis pipeline
classifier = pipeline("sentiment-analysis")

# Take input from the user
print("Enter your text for sentiment analysis (type 'exit' to stop):\n")

while True:
    user_input = input("Your text: ")

    if user_input.lower() == 'exit':
        print("Exiting sentiment analysis.")
        break

    # Analyze the sentiment
    result = classifier(user_input)[0]

    # Print the result
    print(f"\nPredicted Sentiment: {result['label']} | Confidence Score: {result['score']:.4f}")
    print("-" * 60)



No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


Enter your text for sentiment analysis (type 'exit' to stop):

Your text: cozy weather

Predicted Sentiment: POSITIVE | Confidence Score: 0.9997
------------------------------------------------------------
Your text: cute girl

Predicted Sentiment: POSITIVE | Confidence Score: 0.9998
------------------------------------------------------------
Your text: i overslept today

Predicted Sentiment: NEGATIVE | Confidence Score: 0.9970
------------------------------------------------------------
Your text: scored pass mark only

Predicted Sentiment: NEGATIVE | Confidence Score: 0.9332
------------------------------------------------------------
Your text: exit
Exiting sentiment analysis.
